# Results Inspection

In [12]:
import pandas as pd
import glob
import os

pd.set_option('display.max_colwidth', 0)

In [13]:
# origloss_NBTale = pd.read_csv("./logs/OrigLoss_V5/NBTale_results_wav2vec2_NO_origLossV5_2023-09-18.csv")
# origloss_NBTale.drop(columns=["Unnamed: 0", "wav_file", "path", "text"], inplace=True)
# origloss_NBTale.rename(columns={"asr_str": "asr_origloss", "wer":"wer_origloss", "asd":"asd_origloss"}, inplace=True)

In [14]:
path = "./logs/OrigLoss_V5/"
all_files = glob.glob(os.path.join(path , "*.csv"))

origloss_dfs = []
for file in all_files:
    df = pd.read_csv(file)
    df.drop(columns=["Unnamed: 0", "wav_file", "path", "text"], inplace=True)
    df.rename(columns={"asr_str": "asr_origloss", "wer":"wer_origloss", "asd":"asd_origloss"}, inplace=True)
    origloss_dfs.append(df)

origloss = pd.concat(origloss_dfs, ignore_index=True)

In [15]:
# customloss_NBTale = pd.read_csv("./logs/CustomLoss_V7/NBTale_results_wav2vec2_NO_customLossV7_2023-10-02.csv")
# customloss_NBTale.drop(columns=["Unnamed: 0", "wav_file", "path", "text", "segment_id"], inplace=True)
# customloss_NBTale.rename(columns={"asr_str": "asr_customloss", "wer":"wer_customloss", "asd":"asd_customloss"}, inplace=True)

In [16]:
path = "./logs/CustomLoss_V7/"
all_files = glob.glob(os.path.join(path , "*.csv"))

customloss_dfs = []
for file in all_files:
    df = pd.read_csv(file)
    df.drop(columns=["Unnamed: 0", "wav_file", "path", "text", "segment_id"], inplace=True)
    df.rename(columns={"asr_str": "asr_customloss", "wer":"wer_customloss", "asd":"asd_customloss"}, inplace=True)
    customloss_dfs.append(df)

customloss = pd.concat(customloss_dfs, ignore_index=True)

### Merged results of orig loss & custom loss

In [17]:
merged_results = pd.merge(origloss, customloss, on="ref_str")
merged_results = merged_results.reindex(columns=["segment_id", "ref_str", "asr_origloss", "wer_origloss", "asd_origloss", "asr_customloss", "wer_customloss", "asd_customloss"])

,segment_id,ref_str,asr_origloss,wer_origloss,asd_origloss,asr_customloss,wer_customloss,asd_customloss
0,N2_030505_NRK_D12_NO_cut_0,kraftselskapet hafslund blir solgt hvis prisen er god nok sier oslo kommune som ikke vil la seg presse av regjeringen,kraftselskapet hafslund blir solgt hvis prisen er god nok sier oslo kommune som ikke vil la seg presse av regjeringen,0.000000,0.000000,kraftselskapet hafslund blir solgt hvis prisen er god nok sier oslo kommune som ikke vil la seg presse av regjeringen,0.000000,0.000000
1,N2_030505_NRK_D12_NO_cut_1,kvinnen som var savnet i et sjøfly i telemark er funnet omkommet og folk sender gaver til maud angelica men,kvinnen som var savnet i et sjøfly i telemark er funnet omkommet og folk sender gaver til mod angelika men,0.100000,0.123614,kvinnen som var savnet i et sjøfly i telemark er funnet omkommet og folk sender gaver til mod angelika men,0.100000,0.123614
2,N2_030505_NRK_D12_NO_cut_2,hva hun får er hemmelig her er nrk dagsnytt klokka er tolv tretti først til striden rundt kraftselskapet hafslund oslo,hva hun får er hemmelig her er n r k dagsnytt klokka er tolv tretti først til striden rundt kraftselskapet hafslund oslo,0.150000,0.136711,hva hun får er hemmelig her er n r k dagsnytt klokka er tolv tretti først til striden rundt kraftselskapet hafslund oslo,0.150000,0.136711
3,N2_030505_NRK_D12_NO_cut_3,kommune vil ikke la seg presse til å beholde aksjene i selskapet olje og energiministeren vil stanse salget for å,kommune vil ikke la seg presse til å beholde aksjene i selskapet olje og energiministeren vil stanse salget for å,0.000000,0.000000,kommune vil ikke la seg presse til å beholde aksjene i selskapet olje og energiministeren vil stanse salget for,0.050000,0.052995
4,N2_030505_NRK_D12_NO_cut_4,sikre norske eiere i hafslund men finansbyråd i oslo andré støylen lar seg ikke presse og sier at selskapet blir,sikre norske eiere i hafslund men finansbyråd i oslo andre støylen lar seg ikke presse og sier at selskapet blir,0.050000,0.070778,sikre norske eiere i hafslund men finansbyråd i oslo andre støylen lar seg ikke presse og sier at selskapet blir,0.050000,0.070778
...,...,...,...,...,...,...,...,...
824,2021H264-fullStorting0510Stortinget-20210510-115459-Sak1_cut_292,økonomiske utviklingen til kommunen ikke var bærekraftig det er sterke ord i en perspektivmelding resultatet av det var at høyre,økonomiske utviklingen til kommunen ikke var bærekraftig og det er sterke ord i en perspektivmelding resultatet av det var at høyre,0.050000,0.061569,økonomiske utviklingen til kommunen ikke var bærekraftig og det er sterke ord i en perspektivmelding resultatet av det var at høyre,0.050000,0.061569
825,2021H264-fullStorting0510Stortinget-20210510-115459-Sak1_cut_293,frp med frp finansbyråd innførte eiendomsskatt det var ikke noe venstre eller krf i det byrådet som tvang dem til,frpbyrådet med frp finansbyråd innførte eiendomsskatt og det var ikke noe venstre eller krf i det byrådet som tvang de til det det,0.250000,0.229273,frpbyrådet med frp finansbyråd innførte eiendomsskatt og det var ikke noe venstre eller krf i det byrådet som tvang de tider,0.200000,0.234647
826,2021H264-fullStorting0510Stortinget-20210510-115459-Sak1_cut_294,det det var frps finansbyråd frps egen kollega her på stortinget silje hjemdal som i dag er representant på stortinget,frp sin finansbyråd og frp sin egen kollega her på stortinget silje hjemdal som i dag er representant her på stortinget,0.350000,0.278696,frp sin finansbyråd og frp sin egen kollega her på stortinget silje hjemdal som i dag er representant her på stortinget,0.350000,0.278696
827,2021H264-fullStorting0510Stortinget-20210510-115459-Sak1_cut_295,satt i dette byrådet så det er bare å forvente at kollegaene hennes går tilbake fra denne saken og unnskylder,satt i dette byrådet så det er bare til å forvente at kollegaene hennes går tilbake fra denne saken unnskylder,0.100000,0.098308,satt i dette byrådet så det er jo bare til å forvente at kollegaene hennes

In [18]:
better_wer = merged_results[merged_results["wer_origloss"] > merged_results["wer_customloss"]]
worse_wer = merged_results[merged_results["wer_origloss"] < merged_results["wer_customloss"]]
better_asd = merged_results[merged_results["asd_origloss"] > merged_results["asd_customloss"]]
worse_asd = merged_results[merged_results["asd_origloss"] < merged_results["asd_customloss"]]

### better WER but *worse* ASD when using custom loss

In [19]:
better_wer[better_wer["asd_origloss"] < better_wer["asd_customloss"]]

,segment_id,ref_str,asr_origloss,wer_origloss,asd_origloss,asr_customloss,wer_customloss,asd_customloss
11,N2_030505_NRK_D12_NO_cut_11,kan gjøre noe her inne og kor dykk eventuelt vil arrangere meg finansbyråden i oslo måtte svare på mange henvendelser,marineonline finansbyråden i oslo måtte svare på mange henvendelser,0.60,0.492595,kan nå hariris finansbyråden i oslo måtte svare på mange henvendelser,0.55,0.521667
272,N2_050211_NRK_D17_NO_OOLremoved_cut_156,han held fast på at han er uskuldig han er naturligvis fortvilet og skuffet over resultatet straffeutmålingen noterer en at,han held fast på at han er uskuldeghan er naturligvis fortvilet og skuffet over resultatet straffutmålingen noterer en at,0.15,0.098264,han held fast på at han er uskyldig han er naturligvis fortvilet og skuffet over resultatet straffutmålingen noterer en at,0.10,0.123088
397,N2_051206_NRK_D12_NO_OOLremoved_cut_45,i og med at dei har mistillit frå ni av tretten medlemmar i landsrådet så det betyr at spelet er,i og med at de har mistillit fra ni av tretten medlemmer i landsrådet så det betyr at spillet er,0.20,0.109594,i og med at dei har mistillit fra ni av tretten medlemmer i landsrådet så det betyr at spillet er,0.15,0.110652
478,p1_g02_m2_5_x-free_cut_13,jo sånn rett ned fem hundre meter utfor autovernet de andre kom rett etter meg og klart så hva altså,sånn rett ned fem hundre meter utføre ut vernet de andre komme rett etter meg og klarte å så kvasse,0.45,0.490793,sånn rett ned fem hundre meter utføre utover de andre kom rett etter meg og klarte å så kasomskjedd,0.35,0.552101
545,2021H264-fullStorting0510Stortinget-20210510-115459-Sak1_cut_13,eigedomsskatt innan tjue tjuefire eigedomsskatten er ein kommunal skatt og høyrer inn under det kommunale sjølvstyret det er opp til,eigendomsskatt innen tjue tjuefire eiendomsskatten er en kommunal skatt og hører inn under det kommunale sjølstyret det er opp til,0.30,0.172710,eigendomskatt innen tjue tjuefire eiendomsskatten er en kommunal skatt og høyrer inn under det kommunale sjølstyret det er opp til,0.25,0.176661
723,2021H264-fullStorting0510Stortinget-20210510-115459-Sak1_cut_191,renovasjonsavgiften har også hatt et betydelig løft for å få dette til man har hatt gateopprustninger der man har fått,innovasjonssystem renovasjonsavgiften har også hatt et betydelig løft for å få dette til og man har hatt gateopprustninger la andre hvordan også har fått,0.30,0.301255,innovasjonssystem renovasjonsavgiften har også hatt et betydelig løft for å få dette til og man har hatt gateopprustninger klandre hordeogså har fått,0.20,0.383861
821,2021H264-fullStorting0510Stortinget-20210510-115459-Sak1_cut_289,gjør dette som et heltidsverv også helgheim og johnsen og johansen og flere fra frp bør unnskylde til egne kollegaer,gjør dette som et heltidsverv og så bør jo helgheim johnsen johansen og flere fra frp unnskylder til egne kolleger,0.45,0.272333,gjør dette som et heltidsverv og så blir jo helge johnsen og johansen og flere fra frp unnskylder til egne kolleger,0.40,0.277996
825,2021H264-fullStorting0510Stortinget-20210510-115459-Sak1_cut_293,frp med frp finansbyråd innførte eiendomsskatt det var ikke noe venstre eller krf i det byrådet som tvang dem til,frpbyrådet med frp finansbyråd innførte eiendomsskatt og det var ikke noe venstre eller krf i det byrådet som tvang de til det det,0.25,0.229273,frpbyrådet med frp finansbyråd innførte eiendomsskatt og det var ikke noe venstre eller krf i det byrådet som tvang de tider,0.20,0.234647


### worse WER but *better* ASD when using custom loss

In [20]:
worse_wer[worse_wer["asd_origloss"] > worse_wer["asd_customloss"]]

,segment_id,ref_str,asr_origloss,wer_origloss,asd_origloss,asr_customloss,wer_customloss,asd_customloss
9,N2_030505_NRK_D12_NO_cut_9,å sikre et nasjonalt eigarskap det er en debatt som dei nasjonale politikarane får ta eg får forholde meg til,sikre et nasjonalt eierskap det er en debatt som dei nasjonale politikerne får ta for forholde meg til,0.25,0.284005,sikre et nasjonalt eierskap det er en debatt som de nasjonale politikerne får ta jeg for forholde meg til,0.30,0.245522
97,N2_030505_NRK_D12_NO_cut_97,vinglet frem og tilbake i dette spørsmålet og fikk til slutt beskjed av statsminister guy verhofstadt om å godta nattflyvninger,vinglet frem og tilbake i dette spørsmålet og fikk til slutt beskjed av statsminister gifer hofstad om å godta nattflygningr,0.15,0.282468,vinglet frem og tilbake i dette spørsmålet og fikk til slutt beskjed av statsminister gifer hovstad om å godta natt flyvninger,0.20,0.197169
121,N2_050211_NRK_D17_NO_OOLremoved_cut_5,liksom sånn masse forskjellige oster og masse forskjellig melkesort altså ost rømme hva det nå helst måtte være så vil,liksom sånn masse forskjellige oster og masse forskjellig melketår altså ut være,0.50,0.488312,liksom sånn masse forskjellige oster og masse forskjellig melke stort altså ottere,0.60,0.478594
265,N2_050211_NRK_D17_NO_OOLremoved_cut_149,ut i frå det tapet dei føle då tingrettsdommar terje mowatt i dag leste opp drapsdommen på tretten år satt,få det tapet deg føle då tingrett dommer terje mowat i dag leste opp drapsdommen på tretten år satt,0.35,0.314504,ut ifrå det tapet deg føle da tingrett dommer terje mowat i dag leste opp drapsdommen på tretten år sa,0.40,0.239040
302,N2_050211_NRK_D17_NO_OOLremoved_cut_186,kvart å mista lyttarar og dermed inntekter til kanal tjuefire dette er bakgrunnen for at radioselskapa kan bli slått saman,kvart og mista lytter og dermed inntekter til kanal tjuefire dette er bakgrunnen for at radio selskapet kan bli slått sammen,0.25,0.233716,kvart og mistet lyttere og dermed inntekter til kanal tjuefire dette er bakgrunnen for at radio selskapet kan bli slått sammen,0.30,0.220201
402,N2_051206_NRK_D12_NO_OOLremoved_cut_50,med ein viss verdigheit men det er framleis usikkert om ho blir sittande eller ikkje og det får vi ikkje,med en viss verdighet men det er framleis usikkert om hun blir sittende eller ikkje og det får vi ikke,0.25,0.223463,med en viss verdighet men det er framleis usikkert om hun blir sittende eller ikke og det får vi ikke,0.30,0.207827
500,p1_g04_m2_5_x-free_cut_8,til spania så dagen etter altså søndag da satt vi allerede på flyet og landa i madrid sein seint på,til spania så dagen etter altså søndag da satt vi allerede på flyet og landet i madrid sin seint på,0.10,0.103505,til spania så dagen etter altså søndag da satt vi allerede på flyet og landet i madrid sent sent på,0.15,0.079296
524,p1_g08_m2_5_x-free_cut_4,av det mest spennende jeg har gjort er å være tilknyttet senter for borgerkrigsforskning som var ett av de såkalte,det mest spennende jeg har gjort er å være tilknyttet senter for borgerkrigsfrskning som var et av de såkalte,0.15,0.188180,det mest spennende jeg har gjort det å være tilknyttet til senter for borgerkrigsforskning som var et av de såkalte,0.20,0.151398
561,2021H264-fullStorting0510Stortinget-20210510-115459-Sak1_cut_29,på om kommunane kan løysa velferdsoppgåvene sine på ein tilfredsstillande måte eller ikkje med denne regjeringen har vi kuttet skatter,på om kommunene kan løse velferdsoppgavene sine på en tilfredsstillende måte eller ikke ja takk neste taler er representanten aleksander stokkebø høyre deretter ørsal johansen stokkebø ver så god president denne regjeringen har me kuttet skatter,1.20,1.092610,på om kommunene kan løyse velferdsoppgavene sine på en tilfredsstillende måte eller ikke tak takk neste taler er representanten aleksander stoke høyre deretter ørsal johansen stokkebø ver så god president med denne regjeringen har me kutte skatter,1.25,1.068461
666,2021H264-fullStorting0510Stortinge

In [21]:
wer_improved_percent = (len(better_wer)/len(merged_results)) * 100
asd_improved_percent = (len(better_asd)/len(merged_results)) * 100

wer_worsened_percent = (len(worse_wer)/len(merged_results)) * 100
asd_worsened_percent = (len(worse_asd)/len(merged_results)) * 100

In [22]:
print(f"WER improved: {wer_improved_percent:.2f}%")
print(f"ASD improved: {asd_improved_percent:.2f}%")

print(f"WER worsened: {wer_worsened_percent:.2f}%")
print(f"ASD worsened: {asd_worsened_percent:.2f}%")

WER improved: 19.30%
ASD improved: 25.33%
WER worsened: 17.13%
ASD worsened: 24.85%


# CTC LOSS IMPLEMENTATION

In [1]:
import collections
if not hasattr(collections, "Container"):
    import collections.abc
    collections.Container = collections.abc.Container
import transformers
from transformers import AutoTokenizer, BertModel
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor
from transformers import Wav2Vec2ForCTC, Wav2Vec2ProcessorWithLM, TrainingArguments, Trainer
from datasets import load_dataset, load_metric, ClassLabel, Audio, Dataset
import random
import pandas as pd
import math
import numpy as np
import librosa
import os
import torch
from pydub import AudioSegment
from IPython.display import display, HTML
import re
import json
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
import wandb
import argparse
import types
from tabulate import tabulate
from dtw import *
import torch
from scipy.spatial import distance
import copy
import pickle

/localhome/stipendiater/janinelr/miniconda3/envs/wav2vec/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\*]'
def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

def prepare_dataset(batch):
    audio = batch["audio"]
    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [3]:
model_name = "NbAiLab/nb-wav2vec2-300m-bokmaal"

processor = Wav2Vec2ProcessorWithLM.from_pretrained(model_name)
processor_woLM = Wav2Vec2Processor.from_pretrained(model_name)

# model = Wav2Vec2ForCTC.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(
    model_name,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

/localhome/stipendiater/janinelr/miniconda3/envs/wav2vec/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'snapshot_download': allow_regex. Will not be supported from version '0.12'.

Please use `allow_patterns` and `ignore_patterns` instead.
  warnings.warn(message, FutureWarning)
Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 25.29it/s]


In [4]:
# sum(p.numel() for p in model.parameters() if p.requires_grad)

In [5]:
df_train = pd.read_csv("../../model_ckpts/fine-tuning_wav2vec2_customLossV11/runs/train_set.csv")
df_train.drop(columns=["Unnamed: 0"], inplace=True)
train_dataset = Dataset.from_pandas(df_train)
train_dataset = train_dataset.cast_column("path", Audio())
train_dataset = train_dataset.rename_column("path", "audio")
train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16_000))
train_dataset = train_dataset.map(prepare_dataset, remove_columns=train_dataset.column_names, num_proc=4)

#0:   0%|          | 0/5468 [00:00<?, ?ex/s]/localhome/stipendiater/janinelr/miniconda3/envs/wav2vec/lib/python3.10/site-packages/transformers/models/wav2vec2_with_lm/processing_wav2vec2_with_lm.py:584: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/localhome/stipendiater/janinelr/miniconda3/envs/wav2vec/lib/python3.10/site-packages/transformers/models/wav2vec2_with_lm/processing_wav2vec2_with_lm.py:584: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(

/localhome/stipendiater/janinelr/miniconda3/envs/wav2vec/lib/python3.10/site-packages/transformers/

In [6]:
input_feature = [{"input_values": train_dataset[0]["input_values"]}]
label_feature = [{"input_ids": train_dataset[0]["labels"]}]

batch = processor.pad(input_feature, padding=True, return_tensors="pt")

with processor.as_target_processor():
    label = processor.pad(label_feature, padding=True, return_tensors="pt")
label = label["input_ids"].masked_fill(label.attention_mask.ne(1), -100)

batch["labels"] = label

/localhome/stipendiater/janinelr/miniconda3/envs/wav2vec/lib/python3.10/site-packages/transformers/models/wav2vec2_with_lm/processing_wav2vec2_with_lm.py:584: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [7]:
batch

{'input_values': tensor([[-0.1872, -0.0152, -0.0361,  ...,  0.0094,  0.0094,  0.0143]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32), 'labels': tensor([[19,  5, 12, 22,  0, 22,  1, 18,  0, 13,  5,  4,  0, 16, 27,  0, 19, 11,
          1, 16,  5,  0, 18, 21, 14,  4, 20,  0, 13, 21,  8,  1, 13, 13,  5,  4,
         20,  5,  7, 14,  9, 14,  7,  5, 14,  5,  0]])}

In [8]:
output = model(**batch)

In [9]:
labels = batch["labels"]
label_str = processor_woLM.batch_decode(labels, group_tokens=False)
print("REF:", label_str[0])

output_logits = output["logits"].detach().cpu().numpy()
pred_str = processor.batch_decode(output_logits)
print("ASR:", pred_str.text[0])

REF: selv var med på skape rundt muhammedtegningene
ASR: selv var med på å skape rundt muhammedtegningene


### Stanford CTC LOSS

In [3]:
def ctc_loss(params, seq, blank=0):
    seqLen = seq.shape[0]  # length of label sequence
    numphones = params.shape[0]  # number of labels, not even used in the code
    L = 2*seqLen + 1  # length of the label sequence with blanks
    T = params.shape[1]  # length of utterance (time)

    # print(seqLen, numphones, L, T)

    alphas = np.zeros((L,T))
    betas = np.zeros((L,T))

    # convert logits to log softmax
    params = params - np.max(params,axis=0)
    params = np.exp(params)
    params = params / np.sum(params,axis=0)

    # initialize alphas and forward pass
    alphas[0,0] = params[blank,0]
    alphas[1,0] = params[seq[0],0]
    c = np.sum(alphas[:,0])
    alphas[:,0] = alphas[:,0] / c
    llForward = np.log(c)

    for t in range(1,T):
        start = max(0,L-2*(T-t))
        end = min(2*t+2,L)
        for s in range(start,L):
            l = int((s-1)/2)
            # blank
            if s%2 == 0:
                if s==0:
                    alphas[s,t] = alphas[s,t-1] * params[blank,t]
                else:
                    alphas[s,t] = (alphas[s,t-1] + alphas[s-1,t-1]) * params[blank,t]
            # same label twice
            elif s == 1 or seq[l] == seq[l-1]:
                alphas[s,t] = (alphas[s,t-1] + alphas[s-1,t-1]) * params[seq[l],t]
            else:
                alphas[s,t] = (alphas[s,t-1] + alphas[s-1,t-1] + alphas[s-2,t-1]) * params[seq[l],t]

        # normalize at current time (prevent underflow)
        c = np.sum(alphas[start:end,t])
        alphas[start:end,t] = alphas[start:end,t] / c
        llForward += np.log(c)

    # initialize betas and backwards pass
    betas[-1,-1] = params[blank,-1]
    betas[-2,-1] = params[seq[-1],-1]
    c = np.sum(betas[:,-1])
    betas[:,-1] = betas[:,-1] / c
    llBackward = np.log(c)

    for t in range(T-2,-1,-1):
        start = max(0,L-2*(T-t))
        end = min(2*t+2,L)
        for s in range(end-1,-1,-1):
            l = int((s-1)/2)
            # blank
            if s%2 == 0:
                if s == L-1:
                    betas[s,t] = betas[s,t+1] * params[blank,t]
                else:
                    betas[s,t] = (betas[s,t+1] + betas[s+1,t+1]) * params[blank,t]
            # same label twice
            elif s == L-2 or seq[l] == seq[l+1]:
                betas[s,t] = (betas[s,t+1] + betas[s+1,t+1]) * params[seq[l],t]
            else:
                betas[s,t] = (betas[s,t+1] + betas[s+1,t+1] + betas[s+2,t+1]) * params[seq[l],t]

        # normalize at current time
        c = np.sum(betas[start:end,t])
        betas[start:end,t] = betas[start:end,t] / c
        llBackward += np.log(c)

    # Compute gradient with respect to unnormalized input parameters
    grad = np.zeros(params.shape)
    ab = alphas*betas
    for s in range(L):
        l = int((s-1)/2)
        # blank
        if s%2 == 0:
            grad[blank,:] += ab[s,:]
            ab[s,:] = ab[s,:]/params[blank,:]
        else:
            grad[seq[l],:] += ab[s,:]
            ab[s,:] = ab[s,:]/(params[seq[l],:])
    absum = np.sum(ab,axis=0)

    # Check for underflow or zeros in denominator of gradient
    llDiff = np.abs(llForward-llBackward)
    if llDiff > 1e-5 or np.sum(absum==0) > 0:
        print("Diff in forward/backward LL : %f"%llDiff)
        print("Zeros found : (%d/%d)"%(np.sum(absum==0),absum.shape[0]))
        # print(-llForward, grad)
        return (-llForward, grad)
    else:
        grad = params - grad / (params * absum)
        # print(-llForward, grad)
        return (-llForward, grad)

In [2]:
def ctc_loss_tensor(params, seq, blank=0):
    seqLen = seq.shape[0]  # length of label sequence
    L = 2*seqLen + 1  # length of the label sequence with blanks
    T = params.shape[1]  # length of utterance (time)

    alphas = torch.zeros((L,T))
    betas = torch.zeros((L,T))

    # convert logits to log probs
    params = params - (torch.max(params, dim=0)[0])
    params = torch.exp(params)
    params = params / torch.sum(params, dim=0)

    # initialize alphas and forward pass
    alphas[0,0] = params[blank,0]
    alphas[1,0] = params[seq[0],0]
    c = torch.sum(alphas[:,0])
    alphas[:,0] = alphas[:,0] / c
    llForward = torch.log(c)

    for t in range(1,T):
        start = max(0,L-2*(T-t))
        end = min(2*t+2,L)
        for s in range(start,L):
            l = int((s-1)/2)
            # blank
            if s%2 == 0:
                if s==0:
                    alphas[s,t] = alphas[s,t-1] * params[blank,t]
                else:
                    alphas[s,t] = (alphas[s,t-1] + alphas[s-1,t-1]) * params[blank,t]
            # same label twice
            elif s == 1 or seq[l] == seq[l-1]:
                alphas[s,t] = (alphas[s,t-1] + alphas[s-1,t-1]) * params[seq[l],t]
            else:
                alphas[s,t] = (alphas[s,t-1] + alphas[s-1,t-1] + alphas[s-2,t-1]) * params[seq[l],t]

        # normalize at current time (prevent underflow)
        c = torch.sum(alphas[start:end,t])
        alphas[start:end,t] = alphas[start:end,t] / c
        llForward = llForward + torch.log(c)

    # initialize betas and backwards pass
    betas[-1,-1] = params[blank,-1]
    betas[-2,-1] = params[seq[-1],-1]
    c = torch.sum(betas[:,-1])
    betas[:,-1] = betas[:,-1] / c
    llBackward = torch.log(c)

    for t in range(T-2,-1,-1):
        start = max(0,L-2*(T-t))
        end = min(2*t+2,L)
        for s in range(end-1,-1,-1):
            l = int((s-1)/2)
            # blank
            if s%2 == 0:
                if s == L-1:
                    betas[s,t] = betas[s,t+1] * params[blank,t]
                else:
                    betas[s,t] = (betas[s,t+1] + betas[s+1,t+1]) * params[blank,t]
            # same label twice
            elif s == L-2 or seq[l] == seq[l+1]:
                betas[s,t] = (betas[s,t+1] + betas[s+1,t+1]) * params[seq[l],t]
            else:
                betas[s,t] = (betas[s,t+1] + betas[s+1,t+1] + betas[s+2,t+1]) * params[seq[l],t]

        # normalize at current time
        c = torch.sum(betas[start:end,t])
        betas[start:end,t] = betas[start:end,t] / c
        llBackward = llBackward + torch.log(c)

    # Compute gradient with respect to unnormalized input parameters
    grad = torch.zeros(params.shape)
    ab = alphas*betas
    for s in range(L):
        l = int((s-1)/2)
        # blank
        if s%2 == 0:
            grad[blank,:] = grad[blank,:] + ab[s,:]
            ab[s,:] = ab[s,:]/params[blank,:]
        else:
            grad[seq[l],:] = grad[seq[l],:] + ab[s,:]
            ab[s,:] = ab[s,:]/(params[seq[l],:])
    absum = torch.sum(ab,axis=0)

    # Check for underflow or zeros in denominator of gradient
    llDiff = torch.abs(llForward-llBackward)
    if llDiff > 1e-5 or torch.sum(absum==0) > 0:
        print("Diff in forward/backward LL : %f"%llDiff)
        print("Zeros found : (%d/%d)"%(torch.sum(absum==0),absum.shape[0]))
        # print(-llForward, grad)
        return (-llForward, grad)
    else:
        grad = params - grad / (params * absum)
        # print(-llForward, grad)
        return (-llForward, grad)

### Training Trial

In [12]:
model.train()
output = model(**batch)

In [13]:
logits = output["logits"][0].detach().cpu().numpy().transpose()
seq = batch["labels"][0].detach().cpu().numpy()

loss, grad = ctc_loss(logits, seq.squeeze(), blank=0)
print(loss)

267.7602600048259


In [14]:
import torch.nn.functional as F

logits = output["logits"][0]
log_probs = F.log_softmax(logits, dim=-1, dtype=torch.float32)
targets = batch["labels"][0]
input_lengths = torch.tensor(output["logits"][0].shape[0])
target_lengths = torch.tensor(batch["labels"][0].shape[0])

loss = F.ctc_loss(log_probs=log_probs, targets=targets, input_lengths=input_lengths, target_lengths=target_lengths, reduction="sum")
print(loss)

tensor(267.7603, grad_fn=<SumBackward0>)


In [15]:
# torch.autograd.set_detect_anomaly(True)

logits = output["logits"][0].transpose(1,0)
seq = batch["labels"][0]

loss, grad = ctc_loss_tensor(logits, seq, blank=0)
print(loss)
# loss.backward()

# optim = torch.optim.AdamW(model.parameters(), lr=1e-3)
# optim.step()

tensor(267.7602, grad_fn=<NegBackward0>)


# CTC Loss & ASD alignment

In [61]:
logits = output["logits"][0]
label_ids = batch["labels"][0]

pred_ids = np.argmax(logits.detach().numpy(), axis=-1)
logit_frames_decoded = processor.tokenizer.batch_decode(pred_ids, group_tokens=False)
print(logit_frames_decoded)

label_ids = batch["labels"][0]
label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
print(processor.tokenizer.batch_decode(label_ids, group_tokens=False))

['[PAD]', '[PAD]', '', '[PAD]', 's', 's', '[PAD]', '[PAD]', 'e', '[PAD]', 'l', 'l', 'v', 'v', '', '', 'v', 'v', 'a', 'r', 'r', '[PAD]', '', '', 'm', '[PAD]', 'e', '[PAD]', 'd', 'd', '', '', 'p', 'p', 'å', '', '', 'å', 'å', '', '', '', 's', 's', 'k', 'k', '[PAD]', 'a', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 'p', '[PAD]', 'e', '[PAD]', '[PAD]', '', '', 'r', 'r', 'r', 'u', 'n', 'n', '[PAD]', 'd', 'd', 't', '[PAD]', '', '', 'm', 'm', 'e', 'e', 'd', '[PAD]', '', 'h', 'h', '[PAD]', 'a', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 'd', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 't', 't', '[PAD]', '[PAD]', 'e', '[PAD]', 'i', '[PAD]', '[PAD]', '[PAD]', 'n', '[PAD]', 'i', 'i', 'n', '[PAD]', 'g', 'g', 'e', 'e', 'n', 'n', 'e', 'e', 'e', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '']
['s', 'e', 'l', 'v', '', 'v', 'a', 'r', '', 'm', 'e', 'd', '', 'p', 'å', '', 's', 'k', 'a', 'p', 'e', '', 'r', 'u', 'n', 'd', 't', '', 'm', 'u', 'h', 'a', 'm', 'm', 'e', '

In [62]:
# pickle.dump(logit_frames_decoded, open("logit_frames.pkl", "wb"))

torch.Size([130, 34])


In [63]:
ref = processor_woLM.batch_decode(batch["labels"], group_tokens=False)[0]
output_logits = output["logits"].detach().numpy()
hyp = processor.batch_decode(output_logits).text[0]

print("REF:", ref)
print("HYP:", hyp)

REF: selv var med på skape rundt muhammedtegningene
HYP: selv var med på å skape rundt med havteigene


### NOTE: start running here!

In [2]:
ref = "selv var med på skape rundt muhammedtegningene"
hyp = "selv var med på å skape rundt med havteigene"

logit_frames_decoded = pickle.load(open("logit_frames.pkl", "rb"))
logit_frames_decoded

alignment_table = [logit_frames_decoded]
table = tabulate(alignment_table, tablefmt="html")
print("logits decoded")
display(HTML(table))

logits decoded


[PAD],[PAD],,[PAD],s,s,[PAD],[PAD],e,[PAD],l,l,v,v,,,v,v,a,r,r,[PAD],,,m,[PAD],e,[PAD],d,d,,,p,p,å,,,å,å,,,,s,s,k,k,[PAD],a,[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],p,[PAD],e,[PAD],[PAD],,,r,r,r,u,n,n,[PAD],d,d,t,[PAD],,,m,m,e,e,d,[PAD],,h,h,[PAD],a,[PAD],[PAD],[PAD],[PAD],[PAD],d,[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],t,t,[PAD],[PAD],e,[PAD],i,[PAD],[PAD],[PAD],n,[PAD],i,i,n,[PAD],g,g,e,e,n,n,e,e,e,[PAD],[PAD],[PAD],[PAD],[PAD],


In [3]:
modelname = 'ltg/norbert2'
model = BertModel.from_pretrained(modelname)
tokenizer = AutoTokenizer.from_pretrained(modelname)

Some weights of the model checkpoint at ltg/norbert2 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
def get_asd_align(ref, hyp, model, tokenizer):
    tokenized_ref = tokenizer(ref, padding=True, truncation=True, max_length=512, return_tensors="pt")
    tokenized_hyp = tokenizer(hyp, padding=True, truncation=True, max_length=512, return_tensors="pt")
    ref_input_ids = tokenized_ref["input_ids"].squeeze()
    hyp_input_ids = tokenized_hyp["input_ids"].squeeze()

    # print(ref_input_ids)

    with torch.no_grad():
        model_output_ref = model(**tokenized_ref, output_hidden_states=True)
        model_output_hyp = model(**tokenized_hyp, output_hidden_states=True)
    hidden_states_ref = model_output_ref.hidden_states
    hidden_states_hyp = model_output_hyp.hidden_states
    all_layers_reference = []
    all_layers_hypothesis = []
    for i in range(1,13):
        all_layers_reference.append(hidden_states_ref[i].squeeze())
        all_layers_hypothesis.append(hidden_states_hyp[i].squeeze())
    ref_embedding_sequence = torch.stack(all_layers_reference).mean(dim=0)
    hyp_embedding_sequence = torch.stack(all_layers_hypothesis).mean(dim=0)

    alignment = dtw(hyp_embedding_sequence, ref_embedding_sequence, dist_method=distance.cosine, keep_internals=True)
    num_tokens = len(ref_embedding_sequence)
    asd_score = (alignment.distance / num_tokens)

    # print("REF embedding shape:", ref_embedding_sequence.shape)
    # print("HYP embedding shape:", hyp_embedding_sequence.shape)

    ref_alignment_idxs = alignment.index2
    hyp_alignment_idxs = alignment.index1

    # print("REF alignment idxs:", ref_alignment_idxs)
    # print("HYP alignment idxs:", hyp_alignment_idxs)

    cosdist_alignment_tokens = []  # FOR PRINTING
    # ref_alignments = []
    hyp_alignments = []
    for i in range(len(ref_alignment_idxs)):
        ref_embedding = ref_embedding_sequence[ref_alignment_idxs[i]]
        hyp_embedding = hyp_embedding_sequence[hyp_alignment_idxs[i]]
        cosdist = distance.cosine(ref_embedding, hyp_embedding)
        cosdist_alignment_tokens.append(round(cosdist, 3))  # FOR PRINTING
        # ref_token = tokenizer.convert_ids_to_tokens(ref_input_ids[ref_alignment_idxs[i]].reshape(1))[0]
        hyp_token = tokenizer.convert_ids_to_tokens(hyp_input_ids[hyp_alignment_idxs[i]].reshape(1))[0]
        # ref_alignments.append((ref_token, cosdist))
        hyp_alignments.append((hyp_token, cosdist))

    # FOR PRINTING
    ref_alignment_input_ids = np.empty(len(ref_alignment_idxs), dtype=int)
    for i, index in enumerate(ref_alignment_idxs):
        ref_alignment_input_ids[i] = (ref_input_ids[index])
    ref_alignment_tokens = tokenizer.convert_ids_to_tokens(torch.from_numpy(ref_alignment_input_ids))

    hyp_alignment_input_ids = np.empty(len(hyp_alignment_idxs), dtype=int)
    for i, index in enumerate(hyp_alignment_idxs):
        hyp_alignment_input_ids[i] = (hyp_input_ids[index])
    hyp_alignment_tokens = tokenizer.convert_ids_to_tokens(torch.from_numpy(hyp_alignment_input_ids))

    hyp_alignment_tokens.insert(0, "HYP:")
    ref_alignment_tokens.insert(0, "REF:")
    cosdist_alignment_tokens.insert(0, "CosDist:")
    alignment_table = [ref_alignment_tokens, hyp_alignment_tokens, cosdist_alignment_tokens]
    table = tabulate(alignment_table, tablefmt="html")

    print("Token alignment table:")
    display(HTML(table))

    return hyp_alignments

In [5]:
hyp_alignments = get_asd_align(ref, hyp, model, tokenizer)

Token alignment table:


REF:,[CLS],selv,var,med,på,på,skape,rundt,rundt,mu,##ham,##ham,##med,##tegning,##ene,[SEP]
HYP:,[CLS],selv,var,med,på,å,skape,rundt,med,hav,hav,##te,##te,##igen,##e,[SEP]
CosDist:,0.02,0.013,0.014,0.026,0.075,0.331,0.198,0.171,0.687,0.645,0.683,0.697,0.706,0.601,0.265,0.008


In [6]:
for item in hyp_alignments:
    print(item)

('[CLS]', 0.01973193883895874)
('selv', 0.013207316398620605)
('var', 0.013754189014434814)
('med', 0.026312530040740967)
('på', 0.07505130767822266)
('å', 0.33072638511657715)
('skape', 0.19762545824050903)
('rundt', 0.17110878229141235)
('med', 0.686539888381958)
('hav', 0.6446577310562134)
('hav', 0.6833806931972504)
('##te', 0.6969756186008453)
('##te', 0.705551952123642)
('##igen', 0.6006723940372467)
('##e', 0.26462817192077637)
('[SEP]', 0.008164167404174805)


In [7]:
# GROUPING CHARACTERS FROM LOGITS MATRIX INTO THEIR CORRESPONDING WORDS, WHILE RETAINING ROW NOS. OR FRAME NOS.

char_group_list = []
for i, char in enumerate(logit_frames_decoded):
    if i == 0:
        if char == "[PAD]" or char == "":
            continue
    else:
        if char == "[PAD]" or char == "":
            continue
        if len(char_group_list) == 0:  # first word
            if logit_frames_decoded[i-1] in ["", "[PAD]"] and char not in ["", "[PAD]"]:
                new_word = []
                new_word.append((i, char))
                char_group_list.append(new_word)
        else:
            if logit_frames_decoded[i-1] in "" and char not in ["", "[PAD]"]:  # start of the word
                new_word = []
                new_word.append((i, char))
                char_group_list.append(new_word)
            elif logit_frames_decoded[i-1] == char and char != "":  # repeating characters
                new_word.append((i, char))
            elif logit_frames_decoded[i-1] != char and char != "":  # new character detected
                new_word.append((i, char))


for item in char_group_list:
    print(item)

[(4, 's'), (5, 's'), (8, 'e'), (10, 'l'), (11, 'l'), (12, 'v'), (13, 'v')]
[(16, 'v'), (17, 'v'), (18, 'a'), (19, 'r'), (20, 'r')]
[(24, 'm'), (26, 'e'), (28, 'd'), (29, 'd')]
[(32, 'p'), (33, 'p'), (34, 'å')]
[(37, 'å'), (38, 'å')]
[(42, 's'), (43, 's'), (44, 'k'), (45, 'k'), (47, 'a'), (55, 'p'), (57, 'e')]
[(62, 'r'), (63, 'r'), (64, 'r'), (65, 'u'), (66, 'n'), (67, 'n'), (69, 'd'), (70, 'd'), (71, 't')]
[(75, 'm'), (76, 'm'), (77, 'e'), (78, 'e'), (79, 'd')]
[(82, 'h'), (83, 'h'), (85, 'a'), (91, 'd'), (99, 't'), (100, 't'), (103, 'e'), (105, 'i'), (109, 'n'), (111, 'i'), (112, 'i'), (113, 'n'), (115, 'g'), (116, 'g'), (117, 'e'), (118, 'e'), (119, 'n'), (120, 'n'), (121, 'e'), (122, 'e'), (123, 'e')]


In [39]:
# GROUPING THE TOKENS FROM ASD CALCULATION, SUCH THAT WORDPIECES ARE TOGETHER

regrouped_tokens = []
hyp_word_list = hyp.split(" ")  # list of word in the ASR hypothesis
for i, item in enumerate(hyp_alignments):
    if item[0] != "[CLS]" and item[0] != "[SEP]":
        if "##" not in item[0] and "##" in hyp_alignments[i+1][0]:  # start of a group of wordpieces
            wordpiece_group = []
            wordpiece_group.append(item)
            regrouped_tokens.append(wordpiece_group)
        elif "##" in item[0]:  # parts of the word
            wordpiece_group.append(item)
        else:  # not wordpieces
            if item[0] in hyp_word_list:  # omits the inserted starting token of wordpiece group produced by the hyp alignment
                regrouped_tokens.append(item)


for token in regrouped_tokens:
    print(token)

('selv', 0.013207316398620605)
('var', 0.013754189014434814)
('med', 0.026312530040740967)
('på', 0.07505130767822266)
('å', 0.33072638511657715)
('skape', 0.19762545824050903)
('rundt', 0.17110878229141235)
('med', 0.686539888381958)
[('hav', 0.6833806931972504), ('##te', 0.6969756186008453), ('##te', 0.705551952123642), ('##igen', 0.6006723940372467), ('##e', 0.26462817192077637)]


In [38]:
# DIVIDING THE ASD COSDIST PER NO. FRAMES

ASD_cosdist_list = []
for i, char_group in enumerate(char_group_list):
    char_group_copy = copy.deepcopy(char_group)
    # token character list differs when there are whole word tokens or wordpieces
    if isinstance(regrouped_tokens[i], list):
        token_char_list = list(''.join([wordpiece[0].replace("##", "") for wordpiece in regrouped_tokens[i]]))
        token_cosdist = sum([wordpiece[1] for wordpiece in regrouped_tokens[i]])
    else:
        token_char_list = list(regrouped_tokens[i][0])
        token_cosdist = regrouped_tokens[i][1]
    # detecting applicable frames & caclulating average cosdist
    frame_group = []
    for token_char in token_char_list:
        detect_count = 0
        for i, char in enumerate(char_group_copy):
            if char[1] == token_char:
                frame_group.append(char)
                detect_count += 1
            else:
                for count in range(detect_count):
                    char_group_copy.pop(0)
                break
    cosdist_divided = token_cosdist / len(frame_group)  # divide ASD token cosdist by number of frames for word
    # assign divided ASD token cosdist
    frame_cosdist = []
    for char in char_group:
        if char[1] in token_char_list:
            frame_cosdist.append((char[0], char[1], cosdist_divided))
    ASD_cosdist_list.append(frame_cosdist)


for item in ASD_cosdist_list:
    print(item)

[(4, 's', 0.0018867594855172293), (5, 's', 0.0018867594855172293), (8, 'e', 0.0018867594855172293), (10, 'l', 0.0018867594855172293), (11, 'l', 0.0018867594855172293), (12, 'v', 0.0018867594855172293), (13, 'v', 0.0018867594855172293)]
[(16, 'v', 0.002750837802886963), (17, 'v', 0.002750837802886963), (18, 'a', 0.002750837802886963), (19, 'r', 0.002750837802886963), (20, 'r', 0.002750837802886963)]
[(24, 'm', 0.006578132510185242), (26, 'e', 0.006578132510185242), (28, 'd', 0.006578132510185242), (29, 'd', 0.006578132510185242)]
[(32, 'p', 0.02501710255940755), (33, 'p', 0.02501710255940755), (34, 'å', 0.02501710255940755)]
[(37, 'å', 0.16536319255828857), (38, 'å', 0.16536319255828857)]
[(42, 's', 0.02823220832007272), (43, 's', 0.02823220832007272), (44, 'k', 0.02823220832007272), (45, 'k', 0.02823220832007272), (47, 'a', 0.02823220832007272), (55, 'p', 0.02823220832007272), (57, 'e', 0.02823220832007272)]
[(62, 'r', 0.01901208692126804), (63, 'r', 0.01901208692126804), (64, 'r', 0.0